In [ ]:
# !pip install opencv-python

In [ ]:
import os
import time
import uuid
import cv2

In [ ]:
IMAGE_PATH = os.path.join("DataCollection", "images")
OBJECT_CLASS = "test"

OBJECT_PATH = os.path.join(IMAGE_PATH, OBJECT_CLASS)

In [ ]:
# Create images folder
if not os.path.exists(IMAGE_PATH):
    os.mkdir(IMAGE_PATH)

# Create folder for object
if not os.path.exists(OBJECT_PATH):
    os.mkdir(OBJECT_PATH)

## Capture Images from a single camera

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    
    frame = cv2.flip(frame,1)
    cv2.imshow('Capture Images', frame)
    
    key = cv2.waitKey(1) & 0xFF
    # Save an image if 'm' key is pressed
    if key == ord('m'):
        file_name = f'{str(uuid.uuid1())}.jpg'
        img_name = os.path.join(OBJECT_PATH, file_name)
        cv2.imwrite(img_name, frame)

    # Stop image capture if 'q' key is pressed
    if key == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

## Capture Images from multiple cameras

This is only needed if images from multiple cameras should be captured at the same time e.g. RGB Image and Depth Image

In [ ]:
# !pip install vidgear

In [ ]:
RGB_IMAGE_PATH = os.path.join(OBJECT_PATH, "rgb")
DEPTH_IMAGE_PATH = os.path.join(OBJECT_PATH, "depth")

In [ ]:
# Create folder for rgb and depth images
if not os.path.exists(RGB_IMAGE_PATH):
    os.mkdir(RGB_IMAGE_PATH)
    
if not os.path.exists(DEPTH_IMAGE_PATH):
    os.mkdir(DEPTH_IMAGE_PATH)

In [ ]:
from vidgear.gears import VideoGear

# Get multiple video streams
stream1 = VideoGear(source=0, logging=True).start() 
stream2 = VideoGear(source=1, logging=True).start() 

while True:    
    frame1 = stream1.read()
    frame2 = stream2.read()

    if frame1 is None or frame2 is None:
        break
    
    cv2.imshow("Capture Frame1", frame1)
    cv2.imshow("Capture Frame2", frame2)

    key = cv2.waitKey(1) & 0xFF
    # Save an image if 'm' key is pressed
    if key == ord("m"):
        file_name = f'{str(uuid.uuid1())}.jpg'
        rgb_img_name = os.path.join(RGB_IMAGE_PATH, file_name)
        depth_img_name = os.path.join(DEPTH_IMAGE_PATH, file_name)
        
        cv2.imwrite(rgb_img_name, frame1)
        cv2.imwrite(depth_img_name, frame2)
        
    # Stop image capture if 'q' key is pressed
    if key == ord("q"):
        break

cv2.destroyAllWindows()

stream1.stop()
stream2.stop()